In [1]:
import requests
import numpy as np
import pandas as pd
from PIL import Image

import torch
from torch import nn
from torchvision import transforms
from eg3d_dataset import EG3DDataset, EG3DImageProcessor
from gen_samples import vision_evaluate
from torchvision.models import convnext_base


In [2]:
from PIL import Image
import requests
from transformers import BlipProcessor, BlipForConditionalGeneration, BlipTextModel

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(images=image, return_tensors="pt")
# print(inputs.input_ids.shape)

outputs = model(**inputs)

In [3]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 512  # the generated image resolution
    train_batch_size = 16
    eval_batch_size = 16  # how many images to sample during evaluation
    num_dataloader_workers = 12  # how many subprocesses to use for data loading
    num_epochs = 60
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    scheduler_train_timesteps = 1000
    eval_inference_steps = 1000
    save_image_epochs = 10
    save_model_epochs = 10
    mixed_precision = 'no'  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = 'vision-blip-latent-interpreter'
    
    data_dir = 'data_color/'
    df_file = 'dataset.df'

    overwrite_output_dir = True
    seed = 0

config = TrainingConfig()

In [4]:
preprocess = transforms.Compose(
        [
            transforms.Resize(config.image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

dataset = EG3DDataset(df_file=config.df_file, data_dir=config.data_dir, transform=preprocess, encode=False)

train_size = int(len(dataset) * 0.95)
eval_size = len(dataset) - train_size
train_dataset, eval_dataset = torch.utils.data.random_split(dataset, [train_size, eval_size])

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=config.train_batch_size, shuffle=True, num_workers=config.num_dataloader_workers)
eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=config.eval_batch_size, shuffle=True, num_workers=config.num_dataloader_workers)

In [5]:
model = convnext_base()

In [6]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
loss_function = nn.SmoothL1Loss(reduction='mean')

In [7]:
mem_params = sum([param.nelement()*param.element_size() for param in model.parameters()])
mem_bufs = sum([buf.nelement()*buf.element_size() for buf in model.buffers()])
mem = mem_params + mem_bufs
print(mem/(10**9))

0.354365856


In [8]:
from accelerate import Accelerator

from tqdm.auto import tqdm
import os

accelerator = Accelerator(
    mixed_precision=config.mixed_precision,
    gradient_accumulation_steps=config.gradient_accumulation_steps, 
    log_with="tensorboard",
    logging_dir=os.path.join(config.output_dir, "logs")
)
if accelerator.is_main_process:
    accelerator.init_trackers("blip_latent_interpreter")

model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [9]:
def train_loop(config, model, processor, optimizer, train_dataloader, eval_dataloader):
    # Initialize accelerator and tensorboard logging
#     accelerator = Accelerator(
#         mixed_precision=config.mixed_precision,
#         gradient_accumulation_steps=config.gradient_accumulation_steps, 
#         log_with="tensorboard",
#         logging_dir=os.path.join(config.output_dir, "logs")
#     )
#     if accelerator.is_main_process:
#         accelerator.init_trackers("blip_latent_interpreter")

#     model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
#         model, optimizer, train_dataloader, eval_dataloader
#     )

    global_step = 0
    
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")
        model.train()
        
        for step, batch in enumerate(train_dataloader):
            images = batch['images']
            latent_vectors = batch['latent_vectors']
            
            with accelerator.accumulate(model):
                latent_vectors_pred = model(images)
                
                loss = latent_vectors_pred.loss
                accelerator.backward(loss)

                optimizer.step()
                # lr_scheduler.step()
                optimizer.zero_grad()
                
            progress_bar.update(1)
            logs = {"train_loss": loss.detach().item()}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1
        
        model.eval()
        avg_eval_loss = []
        for step, batch in enumerate(eval_dataloader):
            with torch.no_grad():
                latent_vectors_pred = model(pixel_values=images).pooler_output
                
                loss = loss_function(latent_vectors_pred, latent_vectors)
                avg_eval_loss.append(loss.detach().item())
        avg_eval_loss = sum(avg_eval_loss) / len(avg_eval_loss)
        logs = {"eval_loss": avg_eval_loss}
        accelerator.log(logs, step=global_step)

        # After each epoch you optionally sample some demo images with evaluate() and save the model
        if accelerator.is_main_process:
            if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
                render = epoch == config.num_epochs - 1
                vision_evaluate(config, epoch, processor, model, eval_dataloader, render=render)

            if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss,
                }, os.path.join(config.output_dir, 'model.pth'))

In [10]:
from accelerate import notebook_launcher
args = (config, model, processor, optimizer, train_dataloader, eval_dataloader)

notebook_launcher(train_loop, args, num_processes=1)

Launching training on one GPU.


  0%|          | 0/487 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 15.78 GiB total capacity; 14.48 GiB already allocated; 53.12 MiB free; 14.50 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import glob
from IPython.display import display, HTML

sample_dfs = sorted(glob.glob(f"{config.output_dir}/samples/*.df"))
df = pd.read_pickle(sample_dfs[-1])

display(df)